In [15]:
import sys
import os

import asyncio
import random
import time
from test_parallel_local import LocalMockProcessor

In [16]:
async def test_100_jobs_independent_workers(num_workers=10, num_jobs=100):
    """100개 작업을 지정된 워커 수로 독립적으로 처리"""
    print("\n" + "=" * 80)
    print("\n테스트 설정:")
    print(f"   - 총 작업 수: {num_jobs}개")
    print(f"   - 워커 수: {num_workers}개")
    
    test_inputs = [
        {
            "task_name": f"작업_{i+1:03d}",
            "wait_time": round(random.uniform(1, 3), 2),
            "batch": f"batch_{(i//10)+1}",
            "index": i+1
        }
        for i in range(num_jobs)
    ]
    
    total_expected_time = sum(inp["wait_time"] for inp in test_inputs)
    avg_wait_time = total_expected_time / num_jobs
    
    print(f"   - 총 대기 시간: {total_expected_time:.2f}초")
    print(f"   - 평균 대기 시간: {avg_wait_time:.2f}초/작업")
    print(f"   - 순차 처리 예상 시간: ~{total_expected_time:.1f}초")
    print(f"   - 병렬 처리 예상 시간: ~{(total_expected_time/num_workers):.1f}초 (이론적)")
    
    # 프로세서 생성
    processor = LocalMockProcessor(
        base_url="http://localhost:5000",
        num_workers=num_workers
    )
    
    # 독립적 워커 방식으로 처리
    print("\n" + "-" * 80)
    print(f"🔥 {num_workers}개 독립 워커로 {num_jobs}개 작업 처리 시작!")
    print(f"   각 워커는 자기 일이 끝나는 즉시 다음 작업을 시작합니다")
    print("-" * 80)
    print()
    
    start_time = time.time()
    
    # 세마포어로 동시 실행 워커 수 제한
    semaphore = asyncio.Semaphore(num_workers)
    completed_count = [0]  # 완료 카운터 (리스트로 mutable하게)
    lock = asyncio.Lock()  # 출력 동기화용
    
    async def process_with_semaphore(session, input_data, job_index):
        """세마포어를 사용해 워커 수 제한"""
        async with semaphore:
            # 작업 시작
            async with lock:
                print(f"[작업 {job_index+1:3d}] 시작... (워커 할당)")
            
            result = await processor.process_single_job(session, input_data, job_index)
            
            # 작업 완료
            async with lock:
                completed_count[0] += 1
                progress = (completed_count[0] / num_jobs) * 100
                wait_time = result.get("wait_time", 0) if not isinstance(result, Exception) else 0
                print(f"[작업 {job_index+1:3d}] ✅ 완료! (대기: {wait_time:.2f}초) | 진행률: {completed_count[0]}/{num_jobs} ({progress:.1f}%)")
            
            return result
    
    # aiohttp 세션 생성 및 모든 작업 동시 시작
    import aiohttp
    async with aiohttp.ClientSession() as session:
        # 모든 작업을 동시에 시작하지만, 세마포어가 워커 수를 제한
        tasks = [
            process_with_semaphore(session, input_data, idx)
            for idx, input_data in enumerate(test_inputs)
        ]
        
        results = await asyncio.gather(*tasks, return_exceptions=True)
    
    elapsed_time = time.time() - start_time
    
    # 결과 분석
    successful = [r for r in results if not isinstance(r, Exception)]
    failed = [r for r in results if isinstance(r, Exception)]
    
    print("\n" + "=" * 80)
    print("처리 결과")
    print("=" * 80)
    
    print(f"\n성공: {len(successful)}/{num_jobs}개 ({len(successful)/num_jobs*100:.1f}%)")
    print(f"실패: {len(failed)}/{num_jobs}개")
    print(f"\n 총 처리 시간: {elapsed_time:.2f}초")
    print(f"\n 평균 처리 시간: {elapsed_time/num_jobs:.2f}초/작업")
    
    # 속도 비교
    estimated_sequential = total_expected_time
    speedup = estimated_sequential / elapsed_time if elapsed_time > 0 else 0
    efficiency = (speedup / num_workers) * 100
    
    print(f"\n🔥 성능 분석:")
    print(f"   - 예상 순차 처리: {estimated_sequential:.2f}초")
    print(f"   - 실제 병렬 처리: {elapsed_time:.2f}초")
    print(f"   - 속도 향상: {speedup:.2f}배")
    print(f"   - 절약된 시간: {estimated_sequential - elapsed_time:.2f}초 ({(1-elapsed_time/estimated_sequential)*100:.1f}%)")
    print(f"   - 병렬화 효율: {efficiency:.1f}%")
    print(f"   - 이론적 최적: {total_expected_time/num_workers:.2f}초")
    print(f"   - 실제 대비 이론: {(elapsed_time/(total_expected_time/num_workers)):.2f}배 (오버헤드 포함)")
    
    # 처리량 계산
    throughput = num_jobs / elapsed_time
    print(f"\n📊 처리량: {throughput:.2f} 작업/초")
    
    # 결과 저장
    print("\n💾 결과 저장 중...")
    await processor.save_results_to_files(successful, "results_100jobs_independent")
    
    print("\n" + "=" * 80)
    print("✨ 테스트 완료!")
    print("=" * 80)
    
    # 추가 통계
    if successful:
        wait_times = [r.get("wait_time", 0) for r in successful]
        total_times = [r.get("total_time", 0) for r in successful]
        
        print(f"\n📈 상세 통계:")
        print(f"   - 최소 대기 시간: {min(wait_times):.2f}초")
        print(f"   - 최대 대기 시간: {max(wait_times):.2f}초")
        print(f"   - 평균 대기 시간: {sum(wait_times)/len(wait_times):.2f}초")
        print(f"   - 평균 총 처리 시간: {sum(total_times)/len(total_times):.2f}초")
    
    return {
        "num_jobs": num_jobs,
        "num_workers": num_workers,
        "successful": len(successful),
        "failed": len(failed),
        "elapsed_time": elapsed_time,
        "speedup": speedup,
        "efficiency": efficiency,
        "throughput": throughput
    }

In [ ]:
# 워커 수 입력
while True:
    try:
        num_workers_input = input("\n사용할 워커 수를 입력하세요 (기본값: 10, 권장: 5-20): ").strip()
        if num_workers_input == "":
            num_workers = 10
            print(f"→ 기본값 {num_workers}개 워커 사용")
            break
        num_workers = int(num_workers_input)
        if num_workers < 1:
            print("워커 수는 1 이상이어야 합니다.")
            continue

        print(f"→ {num_workers}개 워커로 설정됨")
        break
    except ValueError:
        print("올바른 숫자를 입력하세요.")

try:
    # Jupyter Notebook에서는 asyncio.run() 대신 await 직접 사용
    result = await test_100_jobs_independent_workers(num_workers)
    
    print("\n" + "=" * 80)
    print("🎯 최종 결과 요약")
    print("=" * 80)
    print(f"총 작업: {result['num_jobs']}개")
    print(f"워커 수: {result['num_workers']}개")
    print(f"성공률: {result['successful']}/{result['num_jobs']} ({result['successful']/result['num_jobs']*100:.1f}%)")
    print(f"처리 시간: {result['elapsed_time']:.2f}초")
    print(f"속도 향상: {result['speedup']:.2f}배")
    print(f"병렬화 효율: {result['efficiency']:.1f}%")
    print(f"처리량: {result['throughput']:.2f} 작업/초")
    print("=" * 80)
    
except KeyboardInterrupt:
    print("\n\n⚠️ 테스트가 중단되었습니다.")
except Exception as e:
    print(f"\n❌ 오류 발생: {e}")
    import traceback
    traceback.print_exc()


→ 10개 워커로 설정됨


테스트 설정:
   - 총 작업 수: 100개
   - 워커 수: 10개
   - 총 대기 시간: 197.82초
   - 평균 대기 시간: 1.98초/작업
   - 순차 처리 예상 시간: ~197.8초
   - 병렬 처리 예상 시간: ~19.8초 (이론적)

--------------------------------------------------------------------------------
🔥 10개 독립 워커로 100개 작업 처리 시작!
   각 워커는 자기 일이 끝나는 즉시 다음 작업을 시작합니다
--------------------------------------------------------------------------------

[작업   1] 시작... (워커 할당)
[Worker  1] 작업 제출 중...
[작업   2] 시작... (워커 할당)
[Worker  2] 작업 제출 중...
[작업   3] 시작... (워커 할당)
[Worker  3] 작업 제출 중...
[작업   4] 시작... (워커 할당)
[Worker  4] 작업 제출 중...
[작업   5] 시작... (워커 할당)
[Worker  5] 작업 제출 중...
[작업   6] 시작... (워커 할당)
[Worker  6] 작업 제출 중...
[작업   7] 시작... (워커 할당)
[Worker  7] 작업 제출 중...
[작업   8] 시작... (워커 할당)
[Worker  8] 작업 제출 중...
[작업   9] 시작... (워커 할당)
[Worker  9] 작업 제출 중...
[작업  10] 시작... (워커 할당)
[Worker 10] 작업 제출 중...
[Worker  4] Job ID: ae7e0958... - 대기 중...
[Worker  3] Job ID: 510751ee... - 대기 중...
[Worker  2] Job ID: 6a4b494b... - 대기 중...
[Worker  7] Job ID: 5534253

: 